In [163]:
import pandas as pd
import numpy as np

import sklearn
from sklearn import linear_model

from itertools import cycle
%matplotlib inline
import matplotlib.pyplot as plt
# import statsmodels.api as sm
plt.matplotlib.rcParams['figure.figsize'] = (6,6)

Question 1

(a)

In [164]:
from scipy.stats import norm

def stad_simu(n, periods, K):
    
    # p -> correlation
    # n -> sample size
    # periods -> total periods
    # K -> strike price
    r = .05
    S0 = 100
    sigma = .2
    T = 1
    discFactor = np.exp(-r * T)
    delta = T / float(periods)
    c = np.zeros(n);
    z1 = norm.ppf(np.random.uniform(0, 1, (n, periods)))
    z2 = norm.ppf(np.random.uniform(0, 1, (n, periods)))
    S1 = np.zeros((n, periods))
    S2 = np.zeros((n, periods))
    S1[:, 0] = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z1[:, 0])
    S2[:, 0] = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z2[:, 0])    
    St = (S1[:, -1] + S2[:, -1]) / 2
    c = (St - K) * discFactor
    c[c < 0] = 0
    return (c.mean(), c.std() / np.sqrt(n))

In [165]:
stad_simu(1000, 1, 100)

(8.221230042538842, 0.32489316276468971)

(b)

In [166]:
def strati_binormal(n_sams, n_interv, K):
    r = .05
    S0 = 100
    sigma = .2
    T = 1
    discFactor = np.exp(-r * T)
    delta = T
    intv_len = 1 / float(n_interv)
    num_sam_interval = n_sams / n_interv
    intervals = np.array(zip(np.arange(0, 1, intv_len),
                             np.arange(intv_len, 1 + intv_len, intv_len)))
    results = []
    for inter1 in intervals:
        z1 = norm.ppf(np.random.uniform(inter1[0], inter1[1], num_sam_interval))
        for inter2 in intervals:
            z2 = norm.ppf(np.random.uniform(inter2[0], inter2[1], num_sam_interval))
            S1 = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z1)
            S2 = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z2)
            St = (S1 + S2) / 2
            c = (St - K) * discFactor
            c[c < 0] = 0
            results.append((c.mean(), c.var()))
    result = np.array(zip(*results))
    return(result[0].mean(), np.sqrt(result[1].mean() / n_sams))

In [167]:
strati_binormal(1000, 10, 100)

(8.2578803329530253, 0.094412098186479165)

(c)

In [168]:
def strati_projection(n_sams, n_interv, K):
    r = .05
    S0 = 100
    sigma = .2
    T = 1
    discFactor = np.exp(-r * T)
    delta = T
    nu = np.matrix(np.ones(2) / 2**0.5)
    cov = (np.eye(2) - nu.T * nu).A
    intv_len = 1 / float(n_interv)
    num_sam_interval = n_sams / n_interv
    intervals = np.array(zip(np.arange(0, 1, intv_len),
                             np.arange(intv_len, 1 + intv_len, intv_len)))
    results = []
    for inters in intervals:
        z = np.zeros((2, num_sam_interval))
        for i in range(num_sam_interval):
            x = norm.ppf(np.random.uniform(inters[0], inters[1], 1))
            m = nu.A1 * x
            z[:, i] = np.random.multivariate_normal(m, cov, 1)[0]
        S1 = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z[0,:]) 
        S2 = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z[1,:])    
        St = (S1 + S2) / 2
        c = (St - K) * discFactor
        c[c < 0] = 0
        results.append((c.mean(), c.var()))
    result = np.array(zip(*results))
    return(result[0].mean(), np.sqrt(result[1].mean() / n_sams))

In [169]:
strati_projection(1000, 250, 100)

(8.282420502590238, 0.03163913836119104)

Question 2

(a)

In [225]:
def Euler_disc(n, periods, K):
    r = .1
    sigma = .25
    S0 = 50
    T = .25
    delta = T / float(periods)
    discFactor = np.exp(-r * T)
    w1 = np.random.normal(0, 1, (n, periods))
    St = np.zeros((n, periods))
    St[:, 0] = S0 + r * S0 * delta + np.sqrt(delta) * w1[:, 0] * S0 * sigma
    for i in xrange(1, periods):
        St[:, i] = St[:, i - 1] + r * St[:, i - 1] * delta + np.sqrt(delta) * w1[:, i] * St[:, i - 1] * sigma
    return (St, discFactor)

In [226]:
def standard_simulation(n, periods, K):
    St, discFactor = Euler_disc(n, periods, K)
    Smax = St.max(axis=1)
#     print Smax
    Ct = Smax - K
    Ct[Ct < 0] = 0
    c = discFactor * Ct
    return (c.mean(), c.std() / np.sqrt(n))

In [227]:
standard_simulation(1000, 30, 50)

(5.2096216282716741, 0.14010223616943157)

In [230]:
def brownain_bridge(n, periods, K):
    S0 = 50
    r = .1
    sigma = .25
    S0 = 50
    T = .25
    delta = T / float(periods)
    St, discFactor = Euler_disc(n, periods, K)
    bs = np.zeros((n, periods))
    bs[:, 0] = (St[:, 0] - S0) / (sigma * S0)
    bs[:, 1:] = (St[:, 1:] - St[:, :-1]) / St[:, :-1]
    U = np.random.uniform(0, 1, (n, periods))
    M = (bs + np.sqrt(bs ** 2 - 2 * delta * np.log(1 - U))) / 2
    Smax_Series = np.zeros((n, periods))
    Smax_Series[:, 0] = S0 + sigma * S0 * M[:, 0]
    Smax_Series[:, 1:] = St[:, :-1] + St[:, :-1] * sigma * M[:, 1:]
    Max_S = Smax_Series.max(axis=1)
    sm = St.max(axis=1)
    Ct = Max_S - K
    Ct[Ct < 0] = 0
    c = discFactor * Ct
    return (c.mean(), c.std() / np.sqrt(n))

In [231]:
brownain_bridge(1000, 30, 50)

(5.8751354635881752, 0.14030858950022668)

(b)

In [215]:
def Euler_disc(n, periods):
    r = .1
    sigma = .25
    S0 = 50
    T = .25
    delta = T / float(periods)
    discFactor = np.exp(-r * T)
    w1 = np.random.normal(0, 1, (n, periods))
    St = np.zeros((n, periods))
    St[:, 0] = S0 + r * S0 * delta + np.sqrt(delta) * w1[:, 0] * S0 * sigma
    for i in xrange(1, periods):
        St[:, i] = St[:, i - 1] + r * St[:, i - 1] * delta + np.sqrt(delta) * w1[:, i] * St[:, i - 1] * sigma
    return (St, discFactor)

In [216]:
def standard_simulation_stK(n, periods):
    St, discFactor = Euler_disc(n, periods)
    Smax = St.max(axis=1)
    Ct = Smax - St[:, -1]
    Ct[Ct < 0] = 0
    c = discFactor * Ct
    return (c.mean(), c.std() / np.sqrt(n))

In [217]:
standard_simulation_stK(1000, 30)

(3.7157739746707281, 0.10067136397662017)

In [218]:
def brownain_bridge_stK(n, periods):
    S0 = 50
    r = .1
    sigma = .25
    S0 = 50
    T = .25
    delta = T / float(periods)
    St, discFactor = Euler_disc(n, periods)
    bs = np.zeros((n, periods))
    bs[:, 0] = (St[:, 0] - S0) / (sigma * S0)
    bs[:, 1:] = (St[:, 1:] - St[:, :-1]) / St[:, :-1]
    U = np.random.uniform(0, 1, (n, periods))
    M = (bs + np.sqrt(bs ** 2 - 2 * delta * np.log(1 - U))) / 2
    Smax_Series = np.zeros((n, periods))
    Smax_Series[:, 0] = S0 + sigma * S0 * M[:, 0]
    Smax_Series[:, 1:] = St[:, :-1] + St[:, :-1] * sigma * M[:, 1:]
    Max_S = Smax_Series.max(axis=1)
    sm = St.max(axis=1)
    Ct = Max_S - St[:, -1]
    Ct[Ct < 0] = 0
    c = discFactor * Ct
    return (c.mean(), c.std() / np.sqrt(n))

In [219]:
brownain_bridge_stK(1000, 30)

(4.5928387589227313, 0.10495917585629978)

(c)

In [220]:
def Euler_disc(n, periods, K):
    r = .1
    sigma = .5
    S0 = 50
    T = .25
    delta = T / float(periods)
    discFactor = np.exp(-r * T)
    w1 = np.random.normal(0, 1, (n, periods))
    St = np.zeros((n, periods))
    St[:, 0] = S0 + r * S0 * delta + np.sqrt(delta) * w1[:, 0] * S0 * sigma
    for i in xrange(1, periods):
        St[:, i] = St[:, i - 1] + r * St[:, i - 1] * delta + np.sqrt(delta) * w1[:, i] * St[:, i - 1] * sigma
    return (St, discFactor)

In [221]:
def standard_simulation_MIN(n, periods, K, H):
    St, discFactor = Euler_disc(n, periods, K)
    Smin = St.min(axis=1)
    Ct = St[:, -1] - K
    Ct[(Ct < 0) | (Smin < H)] = 0
    c = discFactor * Ct
    return (c.mean(), c.std() / np.sqrt(n))

In [222]:
standard_simulation_MIN(100000, 30, 50, 45)

(4.5206341436058066, 0.027157662823059151)

In [223]:
def brownain_bridge_MIN(n, periods, K, H):
    S0 = 50
    r = .1
    sigma = .5
    T = .25
    delta = T / float(periods)
    St, discFactor = Euler_disc(n, periods, K)
    bs = np.zeros((n, periods))
    bs[:, 0] = (St[:, 0] - S0) / (sigma * S0)
    bs[:, 1:] = (St[:, 1:] - St[:, :-1]) / St[:, :-1]
    U = np.random.uniform(0, 1, (n, periods))
    M = (bs - np.sqrt(bs ** 2 - 2 * delta * np.log(1 - U))) / 2
    Smin_Series = np.zeros((n, periods))
    Smin_Series[:, 0] = S0 + sigma * S0 * M[:, 0]
    Smin_Series[:, 1:] = St[:, :-1] + St[:, :-1] * sigma * M[:, 1:]
    Min_S = Smin_Series.min(axis=1)
    Ct = St[:, -1] - K
    Ct[(Ct < 0) | (Min_S < H)] = 0
    c = discFactor * Ct
    return (c.mean(), c.std() / np.sqrt(n))

In [224]:
brownain_bridge_MIN(100000, 30, 50, 45)

(4.0409289453206858, 0.02679532290501321)

Question 3

(a)

In [279]:
def correlated_randoms_v(p, n, N):

    # p -> correlation
    # n -> sample size
    # N -> sample dimension
    # vectorized version
    cor_matrix = np.matrix(np.ones((N, N)) * p)
    np.fill_diagonal(cor_matrix, 1)
    cor_decomposed = np.linalg.cholesky(cor_matrix)
    result = np.zeros((N, n))
    r_n = np.matrix(np.random.normal(0, 1, (N, n)))
    copula_transfer = np.array(cor_decomposed * r_n)
    return copula_transfer 

In [280]:
def Euler_disc(n, periods, K):
    r = .1
    sigma = .5
    S0 = 50
    T = .25
    delta = T / float(periods)
    discFactor = np.exp(-r * T)
    w1 = np.random.normal(0, 1, (n, periods))
    St = np.zeros((n, periods))
    St[:, 0] = S0 + r * S0 * delta + np.sqrt(delta) * w1[:, 0] * S0 * sigma
    for i in xrange(1, periods):
        St[:, i] = St[:, i - 1] + r * St[:, i - 1] * delta + np.sqrt(delta) * w1[:, i] * St[:, i - 1] * sigma
    return (St, discFactor)

In [281]:
def stad_simulation_v(p, n, periods, K):
    
    # p -> correlation
    # n -> sample size
    # periods -> total periods
    # K -> strike price
    r = .1
    S0 = 100
    sigma = 0.3
    T = .2
    H = 95
    discFactor = np.exp(-r * T)
    delta = T / float(periods)
    c = np.zeros(n);
    for t in xrange(n):
        prices_paths = np.zeros((2, periods))
        zs = correlated_randoms_v(p, periods, 2)
        prices_paths[:, 0] = S0 + r * S0 * delta + np.sqrt(delta) * zs[:, 0] * S0 * sigma
        for i in xrange(1, periods):
            prices_paths[:, i] = prices_paths[:, i - 1] + r * prices_paths[:, i - 1] * delta +\
                                    np.sqrt(delta) * zs[:, i] * prices_paths[:, i - 1] * sigma
            
        St1 = prices_paths[0, -1]
        St2_min = prices_paths[1, :].min()
        if St1 > K and St2_min > H:
            c[t] = (St1 - K) * discFactor
        
    return (c.mean(), c.std() / np.sqrt(n))

In [282]:
stad_simulation_v(0.5, 10000, 50, 100)

(3.6079428732583745, 0.079913487088217536)

(b)

In [283]:
def Euler_disc_q3(p, periods, K):
    r = .1
    S0 = 100
    sigmas = .3
    T = .2
    H = 95
    delta = T / float(periods)
    discFactor = np.exp(-r * T)
    St = np.zeros((2, periods))
    ws = correlated_randoms_v(p, periods, 2)
    St[:, 0] = S0 + r * S0 * delta + np.sqrt(delta) * ws[:, 0] * S0 * sigmas
    for i in xrange(1, periods):
        St[:, i] = St[:, i - 1] + r * St[:, i - 1] * delta + np.sqrt(delta) * ws[:, i] * St[:, i - 1] * sigmas
    return St

In [286]:
def brownain_bridge_MIN_q3(n, p, periods, K, H):
    S0 = 100
    r = .1
    sigma = .3
    T = .2
    H = 95
    delta = T / float(periods)
    discFactor = np.exp(-r * T)
    c = np.zeros(n)
    for i in xrange(n):
        
        St  = Euler_disc_q3(p, periods, K)
        bs = np.zeros((periods))
        S2 = St[1, :]
        bs[0] = (S2[0] - S0) / (sigma * S0)
        bs[1:] = (S2[1:] - S2[:-1]) / S2[:-1]
        U = np.random.uniform(0, 1, (periods))
        M = (bs - np.sqrt(bs ** 2 - 2 * delta * np.log(1 - U))) / 2
        Smin_Series = np.zeros((periods))
        Smin_Series[0] = S0 + sigma * S0 * M[0]
        Smin_Series[1:] = S2[:-1] + S2[:-1] * sigma * M[1:]
        Min_S = Smin_Series.min()
        if St[0, -1] > K and Min_S > H:
            c[i] = discFactor * (St[0, -1] - K)
    return (c.mean(), c.std() / np.sqrt(n))

In [287]:
brownain_bridge_MIN_q3(10000, 0.5, 50, 100, 95)

(3.1485536798941727, 0.077751596285162816)

Question 4


In [81]:
from scipy.stats import norm

def copula_randoms(p, n, N):

    # p -> correlation
    # n -> sample size
    # N -> sample dimension
    cor_matrix = np.matrix(np.ones((N, N)) * p)
    np.fill_diagonal(cor_matrix, 1)
    cor_decomposed = np.linalg.cholesky(cor_matrix)
    samples = np.zeros((N, n))
    for i in xrange(n):
        r_n = np.matrix(np.random.normal(0, 1, N).reshape(-1, 1))
        copula_transfer = (cor_decomposed * r_n).reshape(1, -1)
        samples[:, i] = copula_transfer
    result = norm.cdf(samples)
    return result 


In [111]:
def sample_result(p, n):

    # p -> correlation
    # n -> sample size
    T = 5
    N = 5
    r = .04
    s = .01
    R = .35
    disc_factor_payoff = np.exp(-r * T) * (1 - R)
    lam = s / (1 - R)
    U_randoms = copula_randoms(p, n, 5)
    Exponential_transfer = -1 / lam * np.log(U_randoms)
    default_times = (Exponential_transfer < T).sum(axis=0)
    ftd  = ((default_times >= 1) * disc_factor_payoff)
    std = ((default_times >= 2) * disc_factor_payoff)
    thdtd = ((default_times >= 3) * disc_factor_payoff)
    fthtd = ((default_times >= 4) * disc_factor_payoff)
    ntd = ((default_times >= 5) * disc_factor_payoff)
    meanS = np.array([ftd.mean(), std.mean(), thdtd.mean(), fthtd.mean(), ntd.mean()])
    StdS = np.array([ftd.std(), std.std(), thdtd.std(), fthtd.std(), ntd.std()]) / np.sqrt(n)
    ixs = ['FTD', '2TD', '3TD', '4TD', 'NTD']
#     colnames = ['mean', 'ste']
    result = pd.DataFrame({'mean': meanS, 'ste': StdS}, index=ixs)
    return result

In [112]:
sample_result(0, 100000)

,mean,ste
FTD,0.169679,0.000784
2TD,0.025385,0.000359
3TD,0.001996,0.000103
4TD,0.000080,0.000021
NTD,0.000000,0.000000


In [113]:
sample_result(0.2, 100000)

,mean,ste
FTD,0.152117,0.000760
2TD,0.036241,0.000424
3TD,0.007498,0.000198
4TD,0.001096,0.000076
NTD,0.000080,0.000021


In [114]:
sample_result(0.4, 100000)

,mean,ste
FTD,0.133182,0.000729
2TD,0.044607,0.000466
3TD,0.015641,0.000284
4TD,0.004523,0.000154
NTD,0.000857,0.000067


In [115]:
sample_result(0.6, 100000)

,mean,ste
FTD,0.112316,0.000687
2TD,0.048912,0.000486
3TD,0.023629,0.000347
4TD,0.010361,0.000233
NTD,0.003310,0.000132


In [116]:
sample_result(0.8, 100000)

,mean,ste
FTD,0.087213,0.000623
2TD,0.049875,0.000490
3TD,0.031180,0.000395
4TD,0.019286,0.000315
NTD,0.009718,0.000225


In [120]:
sample_result(.99999999, 100000)

,mean,ste
FTD,0.039679,0.000442
2TD,0.039663,0.000442
3TD,0.039663,0.000442
4TD,0.039663,0.000442
NTD,0.039663,0.000442
